# Deep active subspaces for the HIV model 

This is a notebook tutorial based on the results of Section 8 of

Edeling, W. (2023). [On the deep active-subspace method](https://doi.org/10.1137/21M1463240). SIAM/ASA Journal on Uncertainty Quantification, 11(1), 62-90.

Here we will apply the deep active subspace method [1] to an HIV model consisting of 7 coupled ordinary differential equations [2], with 27 uncertain input parameters. The equations are:

\begin{align}
\frac{dT}{dt} &= s_1 + \frac{p_1}{C_1+V}TV - \delta_1T - (K_1V + K_2M_I)T,\nonumber\\
\frac{dT_I}{dt} &= \psi(K_1V + K_2M_I)T + \alpha_1T_L-\delta_2T_I-K_3T_ICTL,\nonumber\\
\frac{dT_L}{dt} &= (1-\psi)(K_1V+K_2M_I)T-\alpha_1T_L-\delta_3T_L,\nonumber\\
\frac{dM}{dt} &= s_2+K_4MV-K_5MV-\delta_4M,\nonumber\\
\frac{dM_I}{dt} &= K_5MV-\delta_5M_I-K_6M_ICTL,\nonumber\\
\frac{dCTL}{dt} &= s_3 + (K_7T_I+K_8M_I)CTL-\delta_6CTL,\nonumber\\
\frac{dV}{dt} &= K_9T_I+K_{10}M_I-K_{11}TV-(K_{12}+K_{13})MV-\delta_7V,
\end{align}

The 27 input parameters are **prescribed uniform distributions with boundaries set at $\pm$ 2.5\% of their nominal values** (see the Supplementary Materials of the article above for specific values, and for a description of the ODE variables). We just note that our (scalar) quantity of interest is the T-cell count $T(t)$ at a given time, and we refer to [2] for further information on the model.

[1] Tripathy, R., & Bilionis, I. (2019, August). Deep active subspaces: A scalable method for high-dimensional uncertainty propagation. In International Design Engineering Technical Conferences and Computers and Information in Engineering Conference (Vol. 59179, p. V001T02A074). American Society of Mechanical Engineers.

[2] Loudon, T., & Pankavich, S. (2017). Mathematical analysis and dynamic active subspaces for a long term model of HIV. Mathematical Biosciences and Engineering, 14(3), 709-733.

### Requirements

We will use a standard ANN to find the active subspace, using [EasySurrogate](https://github.com/wedeling/EasySurrogate). To install, simply uncomment the `!pip install` line below. Furthermore, `scipy` and `matplotlib` are also required.

In [ ]:
# !pip install easysurrogate
# !pip install scipy
# !pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import easysurrogate as es
from scipy import linalg

In [ ]:
# select the seismic color scheme
plt.rcParams['image.cmap'] = 'seismic'

### Choose initial dimension active subspace

We must begin by simply guessing a value for the dimension of the active subspace: $d > 1$:

In [ ]:
########################################
# choose the active subspace dimension #
########################################
d = 5

### Compute the active subspace

Our goal is to look for eigenvalue decay in 

\begin{align}
 C_{ref} = \mathbb{E}\left[\left(\nabla f\left({\bf x}\right)\right)\left(\nabla f\left({\bf x}\right)\right)^T\right] = \int \left(\nabla f\left({\bf x}\right)\right)\left(\nabla f\left({\bf x}\right)\right)^T p(\bf x)d{\bf x}.
\end{align}

Here $p(\bf x)$ is the joint input probability density function. $C_{ref}$ requires access to the code gradient, which we'll assume we do not have. Instead we will approximate $C_{ref}$ using machine learning.

### EasySurrogate campaign

EasySurrogate's basic object is called a `campaign', which handles the data. 

In [ ]:
# Create EasySurrogate campaign
campaign = es.Campaign()

### Load training data

Here we use the campaign to load the training data, which is precomputed and stored in the `my_samples.hdf5` file. 

In [ ]:
##########################
# Generate training data #
##########################

# number of inputs
D = 27

# the times (in days) at which the HIV model was sampled
times = np.array([5, 15, 24, 38, 40, 45, 50, 55, 65, 90, 140, 500, 750,
                  1000, 1600, 1800, 2000, 2200, 2400, 2800, 3400])
T = times.size

# Load HDF5 training data using the Campaign
data = campaign.load_hdf5_data(file_path='my_samples.hdf5')
# input parameters in [-1, 1]
params = data['inputs']
# output (T-cell counts at times)
samples = data['outputs']

# time index at which to construct an active subspace
I = -1
samples = samples[:, I].reshape([-1, 1])

### Train a deep active subspace network

Below we train a our neural network. Here:

* `params`: are the uniformly distributed input variables
* `samples`: are the corresponding HIV code outputs
* `n_iter`: the number of mini batch iterations
* `n_layers`: number of layers, not counting the input layer.
* `activation = ['linear', 'tanh', 'tanh']`: are the activations of the hidden layers. The DAS layer, is a linear encoder, such that the first activation is linear. The output layer is automatically assigned a linear activation.
* `batch_size`: the size of a single mini batch
* `standardize_X=False`: do not standardize the inputs, as they are already scaled to [-1,1]
*  `standardize_y=True`: do standardize the outputs.

In [ ]:
#####################
# train DAS network #
#####################

das_surrogate = es.methods.ANN_Surrogate()
das_surrogate.train(params, samples, n_iter=10000, 
                    n_layers=4,
                    activation = ['linear', 'tanh', 'tanh'],
                    n_neurons=[d, 50, 50],
                    batch_size = 64, 
                    standardize_X=False, standardize_y=True)

### Compute the original active subspace of the DAS network

**Assignment**: compute the DAS equivalent of the reference gradient matrix `C_ref`, i.e. approximate compute:

\begin{align}
 C_{DAS} = \int \left(\nabla\tilde{f}(\bf x)\right)\left(\nabla\tilde{f}(\bf x)\right)^T p({\bf x})d{\bf x}
\end{align}

using Monte Carlo sampling. Here $\nabla\tilde{f}(\bf x)$ is the derivative of the neural network, computed via `das_surrogate.derivative`.

In [ ]:
# the gradient matrix computed of the DAS network, computed using the classical AS method
C_das = 0.0

# Compute C1 and C_das
n_mc = params.shape[0]
das_samples = np.zeros(n_mc)
for i, param in enumerate(params):
    # compute the derivative of f at the input layer (needed for C_das)
    df_dx = das_surrogate.derivative(param, norm=False)
    # store predictions for later
    das_samples[i] = das_surrogate.predict(param)
    # update C_das below
    ....

**Assigment**: compute the eigenvalues and eigenvectors of `C_das` using `linalg.eigh` and sort them according to eigenvalue magnitude:

In [ ]:
# solve eigenvalue problem for C_das
eigvals_C_das, eigvecs_C_das = ...

**Assigment**: extract the $d$ dominant eigenvectors, and compute the active variable $y = W_1^T{\bf x}$ for all ${\bf x}$ values from the training data (`params`):

In [ ]:
# the d dominant eigenvectors
W_1 = 

### Recreate the eigenvalue plots

Compare the eigenvalue decay of `C_DAS`. Is there an active subspace, and if yes what is its dimension?

In [ ]:
####################
# plot eigenvalues #
####################



### Create the active subspace plot

**Assignment**: Plot the HIV output (`das_samples`) in the active subspace, and also plot some validation data by running `HIV_model.py` at some random paramater values.

In [ ]:
# Generate new code validation samples
from HIV_model import *
n_val = 100
x_val = np.random.rand(n_val, D) * 2 - 1
val_samples = Tcells(x_val, np.linspace(1, times[I], times[I]))[:, -1]

In [ ]:
# active subspace coordinates of validation data
y_val = np.dot(W_1.T, x_val.T).T

# plot DAS surrogate predicion and validation data in y coordinate



### Global-derivative based sensitivity plots

To estimate the contribution of individual paramater values, we compute the global derivative-based sensitivity indices:

\begin{align}
\nu_i :=\int\left(\frac{\partial \tilde{f}}{\partial x_i}\right)^2p\left({\bf x}\right)\mathrm{d}{\bf x}.
\end{align}

Just run the cells below to compute this.

In [ ]:
def sensitivity(idx, V_i, **kwargs):
    """
    Plot the sensitivity indices.
    
    idx : array
        Indices of the sensitivity indices, ranking them from most inportant to least important.
    
    V_i : array
        The sensitivity indices.

    """
    
    # Parameter names
    param_names = np.array([r'$s_1$', r'$s_2$', r'$s_3$', r'$p_1$', r'$C_1$', r'$K_1$', r'$K_2$', r'$K_3$',
                   r'$K_4$', r'$K_5$', r'$K_6$', r'$K_7$', r'$K_8$', r'$K_9$', r'$K_{10}$',
                   r'$K_{11}$', r'$K_{12}$', r'$K_{13}$', r'$\delta_1$', r'$\delta_2$',
                   r'$\delta_3$', r'$\delta_4$', r'$\delta_5$', r'$\delta_6$', r'$\delta_7$', r'$\alpha_1$',
                   r'$\psi$'])
    
    fig = plt.figure(figsize=[4, 8])
    ax = fig.add_subplot(111, title=kwargs.get('title', ''))
    # ax.set_ylabel(r'$\int\left(\frac{\partial f}{\partial x_i}\right)^2 p({\bf x})d{\bf x}$', fontsize=14)
    ax.set_xlabel(r'$\nu_i$', fontsize=14)    
    ax.barh(range(V_i.size), width = V_i[idx].flatten(), color = 'dodgerblue')
    ax.set_yticks(range(V_i.size))
    ax.set_yticklabels(param_names[idx[0]], fontsize=14)
    # plt.xticks(rotation=90)
    ax.invert_yaxis()
    sns.despine(top=True)
    plt.tight_layout()

In [ ]:
#####################################
# global gradient-based sensitivity #
#####################################

das_analysis = es.analysis.DAS_analysis(das_surrogate)

idx, V_i = das_analysis.sensitivity_measures(params, norm=False)
sensitivity(idx, V_i, title = 'DAS')